In [4]:
import pandas as pd
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())


In [5]:
pd.set_option("display.width",70)

In [6]:
data = pd.read_csv('./police.csv')
data.head()

,stop_date,stop_time,county_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop
0,2005-01-02,01:55,NaN,M,1985.0,20.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
1,2005-01-18,08:15,NaN,M,1965.0,40.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
2,2005-01-23,23:15,NaN,M,1972.0,33.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
3,2005-02-20,17:15,NaN,M,1986.0,19.0,White,Call for Service,Other,False,NaN,Arrest Driver,True,16-30 Min,False
4,2005-03-14,10:00,NaN,F,1984.0,21.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False


In [7]:
data.shape

(91741, 15)

In [37]:
data.dtypes

stop_date              object
stop_time              object
driver_gender          object
driver_age_raw        float64
driver_age            float64
driver_race            object
violation_raw          object
violation              object
search_conducted         bool
search_type            object
stop_outcome           object
is_arrested            object
stop_duration          object
drugs_related_stop       bool
dtype: object

In [8]:
data.isnull().sum()

stop_date                 0
stop_time                 0
county_name           91741
driver_gender          5335
driver_age_raw         5327
driver_age             5621
driver_race            5333
violation_raw          5333
violation              5333
search_conducted          0
search_type           88545
stop_outcome           5333
is_arrested            5333
stop_duration          5333
drugs_related_stop        0
dtype: int64

In [9]:
data.drop("county_name",axis = 1,inplace = True)

In [10]:
data["driver_gender"].value_counts()

M    62895
F    23511
Name: driver_gender, dtype: int64

In [11]:
data[data["driver_gender"] == "F"]["driver_age"].mean()

32.60739856801909

In [12]:
print("{0:%}".format(round((data[data["driver_race"]=="White"].shape[0]/data.shape[0]),2)))

68.000000%


In [13]:
ages1 = data[data["violation"] == "Speeding"]["driver_age"]
ages2 = data[data["violation"] == "Equipment"]["driver_age"]
print("Speeding : = {0} ± {1} years".format(ages1.mean(), ages1.std()))
print("Equipment : = {0} ± {1} years".format(ages2.mean(), ages2.std()))


Speeding : = 33.53009656541428 ± 12.821847021840082 years
Equipment : = 31.781502680112656 ± 11.400899609138921 years


In [14]:
 out_result = set(["Arrest Driver", "Arrest Passenger"])
def out_resulted(e):return e in out_result 

data[data["is_arrested"] == "True"]["stop_outcome"].map(out_resulted).all()



True

In [91]:
data.groupby(["driver_race", "driver_gender"])["driver_age"].describe()

count       mean        std   min  \
driver_race driver_gender                                        
Asian       F                511.0  31.996086  10.847245  17.0   
            M               1742.0  34.044202  11.886878  17.0   
Black       F               2577.0  31.008925  10.216993  17.0   
            M               9620.0  33.673493  11.748992  15.0   
Hispanic    F               1867.0  30.660418  10.007572  16.0   
            M               7610.0  31.787385  10.682480  15.0   
Other       F                 26.0  30.923077  11.520150  20.0   
            M                213.0  34.990610  11.567681  18.0   
White       F              18483.0  33.046205  12.592247  15.0   
            M              43464.0  35.228350  13.470667  15.0   

                             25%   50%   75%   max  
driver_race driver_gender                           
Asian       F              23.00  29.0  38.5  75.0  
            M              24.25  31.0  42.0  82.0  
Black       F              23.00  28.0  37.0  78.0  
            M              24.00  31.0  41.0  85.0  
Hispanic    F              23.00  28.0  36.0  71.0  
            M              23.00  29.0  38.0  76.0  
Other       F              22.25  26.5  35.0  60.0  
            M              25.00  33.0  42.0  74.0  
White       F              23.00  29.0  42.0  99.0  
            M              24.00  32.0  45.0  94.0

In [47]:
data["ince_rest"] = data["search_type"].str.startswith("Incident to Arrest")


data[(data["driver_gender"] == "M")]["ince_rest"].value_counts()



False    1419
True     1306
Name: ince_rest, dtype: int64

In [15]:
m = data["driver_age"].min() 
print("Minimum is {} age.".format(m))
people = data[data["driver_age"] == m]
c = people.shape[0]
print("{0} people with age {1} checked by polic.".format(c,m))
s = people[people["driver_race"] == "White"].shape[0] 
print("{0:%} whilte people.".format(s / c))


Minimum is 15.0 age.
5 people with age 15.0 checked by polic.
60.000000% whilte people.


In [16]:
pd.crosstab(data["violation"], data["is_arrested"])



is_arrested,False,True
violation,,
Equipment,10385,635
Moving violation,15314,910
Other,3978,339
Registration/plates,3098,334
Seat belt,2878,74
Speeding,47826,637


In [17]:
p=pd.crosstab(data["violation"], data["is_arrested"],values=data['driver_age'], aggfunc="mean")
p

is_arrested,False,True
violation,,
Equipment,31.819900,31.154331
Moving violation,36.298683,33.114664
Other,40.110709,32.973373
Registration/plates,32.964759,31.305389
Seat belt,32.242877,30.783784
Speeding,33.571914,30.397174


In [18]:
p.loc[("Speeding")]

is_arrested
False    33.571914
True     30.397174
Name: Speeding, dtype: float64

In [19]:
dc=pd.read_csv("./dc-wikia-data.csv")
marvel = pd.read_csv("./marvel-wikia-data.csv")

In [20]:
dc.head()

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,YEAR
0,1422,Batman (Bruce Wayne),\/wiki\/Batman_(Bruce_Wayne),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3093.0,"1939, May",1939.0
1,23387,Superman (Clark Kent),\/wiki\/Superman_(Clark_Kent),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2496.0,"1986, October",1986.0
2,1458,Green Lantern (Hal Jordan),\/wiki\/Green_Lantern_(Hal_Jordan),Secret Identity,Good Characters,Brown Eyes,Brown Hair,Male Characters,NaN,Living Characters,1565.0,"1959, October",1959.0
3,1659,James Gordon (New Earth),\/wiki\/James_Gordon_(New_Earth),Public Identity,Good Characters,Brown Eyes,White Hair,Male Characters,NaN,Living Characters,1316.0,"1987, February",1987.0
4,1576,Richard Grayson (New Earth),\/wiki\/Richard_Grayson_(New_Earth),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,1237.0,"1940, April",1940.0


In [21]:
marvel.head()

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,Year
0,1678,Spider-Man (Peter Parker),\/Spider-Man_(Peter_Parker),Secret Identity,Good Characters,Hazel Eyes,Brown Hair,Male Characters,NaN,Living Characters,4043.0,Aug-62,1962.0
1,7139,Captain America (Steven Rogers),\/Captain_America_(Steven_Rogers),Public Identity,Good Characters,Blue Eyes,White Hair,Male Characters,NaN,Living Characters,3360.0,Mar-41,1941.0
2,64786,"Wolverine (James \""Logan\"" Howlett)",\/Wolverine_(James_%22Logan%22_Howlett),Public Identity,Neutral Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3061.0,Oct-74,1974.0
3,1868,"Iron Man (Anthony \""Tony\"" Stark)",\/Iron_Man_(Anthony_%22Tony%22_Stark),Public Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2961.0,Mar-63,1963.0
4,2460,Thor (Thor Odinson),\/Thor_(Thor_Odinson),No Dual Identity,Good Characters,Blue Eyes,Blond Hair,Male Characters,NaN,Living Characters,2258.0,Nov-50,1950.0


In [22]:
dc_filter = dc.drop(['GSM'], axis = 1).dropna()
marvel_filter = marvel.drop(['GSM'],axis=1).dropna()

In [23]:
item=pd.read_csv("./items.csv")
sales = pd.read_csv("./sales_train.csv")

In [24]:
item.dtypes

item_id             int64
item_category_id    int64
dtype: object

In [25]:
sales.dtypes

date               object
date_block_num      int64
shop_id             int64
item_id             int64
item_price        float64
item_cnt_day        int64
dtype: object

In [26]:
sales.merge(item[["item_id","item_category_id"]],on="item_id").head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id
0,12.01.2013,0,25,19,28.0,1,40
1,02.01.2013,0,19,27,2499.0,1,19
2,09.01.2013,0,26,27,2499.0,1,19
3,11.01.2013,0,2,27,2499.0,1,19
4,12.01.2013,0,1,27,1890.0,1,19


In [28]:
%%timeit
sales.merge(item[["item_id","item_category_id"]],on="item_id")



5.4 ms ± 250 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [30]:
 pysqldf("""SELECT s.date, s.date_block_num, s.shop_id,s.item_id,
s.item_id, s.item_id,s.item_cnt_day,t.item_category_id  FROM sales AS s JOIN item AS t
ON s.item_id = t.item_id""").head()


,date,date_block_num,shop_id,item_id,item_id,item_id,item_cnt_day,item_category_id
0,12.01.2013,0,25,19,19,19,1,40
1,02.01.2013,0,19,27,27,27,1,19
2,04.01.2013,0,28,29,29,29,1,23
3,09.01.2013,0,26,27,27,27,1,19
4,11.01.2013,0,2,27,27,27,1,19


In [31]:
%%timeit
 pysqldf("""SELECT s.date, s.date_block_num, s.shop_id,s.item_id,
s.item_id, s.item_id,s.item_cnt_day,t.item_category_id  FROM sales AS s JOIN item AS t
ON s.item_id = t.item_id""")

27.1 ms ± 609 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [32]:
sales.groupby("shop_id")["item_price"].mean().head()

shop_id
1    1890.0
2    2499.0
3     249.0
4     398.0
5    1298.0
Name: item_price, dtype: float64

In [33]:
%%timeit
sales.groupby("shop_id")["item_price"].mean().head()

893 µs ± 12.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [34]:
pysqldf("""SELECT shop_id, AVG(item_price)
FROM sales
GROUP BY shop_id""").head()


,shop_id,AVG(item_price)
0,1,1890.0
1,2,2499.0
2,3,249.0
3,4,398.0
4,5,1298.0


In [35]:
%%timeit
pysqldf("""SELECT shop_id, AVG(item_price)
FROM sales
GROUP BY shop_id""").head()

13.6 ms ± 334 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
